In [ ]:
import requests
import json
import csv

In [ ]:
# Authenticate with the Spotify API using the Authorization Code Flow

auth_url = 'https://accounts.spotify.com/authorize'
client_id = 'your_client_id_here'
redirect_uri = 'http://localhost:8000/callback'
scope = 'select prefered scopes from here, https://developer.spotify.com/documentation/web-api/concepts/scopes'

auth_params = {
    'client_id': client_id,
    'response_type': 'code',
    'redirect_uri': redirect_uri,
    'scope': scope
}

auth_url_params = '&'.join([f"{key}={val}" for key, val in auth_params.items()])
auth_url = f"{auth_url}?{auth_url_params}"

print("Go to the following URL to authorize the app:")
print(auth_url)

In [ ]:
# After authorization, retrieve the access token using the authorization code

auth_code = input("Enter the authorization code: ")
access_token_url = 'https://accounts.spotify.com/api/token'

access_token_params = {
    'client_id': client_id,
    'client_secret': 'your_client_secret_here',
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': redirect_uri
}

access_token_response = requests.post(access_token_url, data=access_token_params)
access_token = json.loads(access_token_response.text)['access_token']


In [ ]:
# Use the access token to gather user data

headers = {
    'Authorization': f'Bearer {access_token}'
    
}

In [ ]:
# Get the user's favorite artists

favorite_artists_url = 'https://api.spotify.com/v1/me/playlists'
favorite_artists_response = requests.get(favorite_artists_url, headers=headers)
favorite_artists = json.loads(favorite_artists_response.text)
print(favorite_artists)

In [ ]:
# Set the URL for the Spotify API request
saved_tracks_url = 'https://api.spotify.com/v1/me/tracks'


# Set the parameters for the Spotify API request
params = {
    'limit': 50,
    'offset': 0
}

# Send the Spotify API request and convert the response data to a Python dictionary
saved_tracks_response = requests.get(saved_tracks_url, headers=headers, params=params)
saved_tracks_data = json.loads(saved_tracks_response.text)

# Print the names and artists of the saved tracks
for track in saved_tracks_data['items']:
    print(f"Track Name: {track['track']['name']}")
    print(f"Artist Name: {track['track']['artists'][0]['name']}")
    print(f"Track Popularity:  {track['track']['popularity']}")
    print(f"Track Duration:  {(track['track']['duration_ms'])/60000}")
    print()

In [ ]:
# Define the field names for the CSV file
field_names = ['Track Name', 'Artist Name', 'Track Popularity', 'Track Duration in mins']

# Open a new CSV file and write the header row
with open('Spotify_tracks.csv', mode='w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=field_names)
    writer.writeheader()

    # Write each track and artist to a new row in the CSV file
    for track in saved_tracks_data['items']:
        writer.writerow({'Track Name': track['track']['name'], 'Artist Name': track['track']['artists'][0]['name'], 'Track Popularity' : track['track']['popularity'], 'Track Duration in mins' :  (track['track']['duration_ms'])/60000})

print('Data written to CSV file.')

In [ ]:

# Set the URL for the Spotify API request
new_releases_url = 'https://api.spotify.com/v1/browse/new-releases'



# Set the parameters for the Spotify API request
params = {
    'limit': 50
}

# Send the Spotify API request and convert the response data to a Python dictionary
new_releases_response = requests.get(new_releases_url, headers=headers, params=params)
new_releases_data = json.loads(new_releases_response.text)

# Open a CSV file for writing and write the header row
with open('new_spotify_releases.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Album Name', 'Artist Name', 'Release Date', 'Album Artwork URL'])

    # Write each album's data to the CSV file
    for album in new_releases_data['albums']['items']:
        album_name = album['name']
        artist_name = album['artists'][0]['name']
        release_date = album['release_date']
        album_artwork_url = album['images'][0]['url']
        writer.writerow([album_name, artist_name, release_date, album_artwork_url])
        print(f"Album Name: {album_name}")
        print(f"Artist Name: {artist_name}")
        print(f"Release Date: {release_date}")
        print(f"Album Artwork URL: {album_artwork_url}")
        print()

print("Data saved to new_spotify_releases.csv")